In [2]:
import ee
import geemap
import sys,os
sys.path.append(r'D:\09_Code\Gis_Script\GEE_Func')
from S1_distor_dedicated import load_S1collection,S1_CalDistor,DEM_caculator
from S2_filter import merge_s2_collection
from GEE_DataIOTrans import BandTrans,DataTrans,DataIO
import math
from functools import partial

geemap.set_proxy(port=10809)
ee.Initialize()
# ee.Authenticate()

# 几何畸变矫正与融合

In [3]:
def CalDitor(s1_ascending,s1_descending,Bands_ = ['VV_gamma0_flatDB','VH_gamma0_flatDB'],Origin_Method='RS',Method:str='1',AOI_buffer=None,Origin_scale=10,projScale=30): 
    volumetric_dict = DEM_caculator.My_s1_slope_correction(s1_ascending,s1_descending,AOI_buffer,DEMNASA,Model,Origin_scale,DistorMethed=Origin_Method)

    # ----------------------Z-scor均衡化，直方图匹配，在局部进行直方图匹配
    Ascending_Img = volumetric_dict['ASCENDING'].select(Bands_)
    Descending_Img = volumetric_dict['DESCENDING'].select(Bands_)
    A_median = Ascending_Img.reduceRegion(**{
                          'reducer': ee.Reducer.median(),
                          'geometry':AOI_buffer
                        }).toArray()
    D_median = Descending_Img.reduceRegion(**{
                          'reducer': ee.Reducer.median(),
                          'geometry':AOI_buffer
                        }).toArray()
    Ascending_Img = Ascending_Img.subtract(A_median.subtract(D_median).getInfo())
    volumetric_dict['ASCENDING'] = BandTrans.replaceBands(volumetric_dict['ASCENDING'],Ascending_Img)


    # -----------------------基于线性关系。检测几何畸变
    Templist_A = S1_CalDistor.AuxiliaryLine2Point(volumetric_dict['ASCENDING'],volumetric_dict['ASCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['ASCENDING_parms']['coordinates_dict'],
                                volumetric_dict['ASCENDING_parms']['Auxiliarylines'],
                                projScale)

    Templist_D = S1_CalDistor.AuxiliaryLine2Point(volumetric_dict['DESCENDING'],volumetric_dict['DESCENDING_parms']['s1_azimuth_across'],
                                volumetric_dict['DESCENDING_parms']['coordinates_dict'],
                                volumetric_dict['DESCENDING_parms']['Auxiliarylines'],
                                projScale)
    if   Method == '1':
        LeftLayoverA,RightLayoverA,ShadowA = S1_CalDistor.Line_Correct(volumetric_dict['ASCENDING'],AOI_buffer,Templist_A,'ASCENDING',
                                                volumetric_dict['ASCENDING_parms']['proj'],projScale,Origin_scale,
                                                filt_distance=False, save_peak=False, line_points_connect=True,Peak_Llay=True,Peak_shdow=True,Peak_Rlay=True)
        LeftLayoverD,RightLayoverD,ShadowD =  S1_CalDistor.Line_Correct(volumetric_dict['DESCENDING'],AOI_buffer,Templist_D,'DESCENDING',
                                                volumetric_dict['DESCENDING_parms']['proj'],projScale,Origin_scale,
                                                filt_distance=False,save_peak=False,line_points_connect=True,Peak_Llay=True,Peak_shdow=True,Peak_Rlay=True)
    elif Method == '2':
        LeftLayoverA,RightLayoverA,ShadowA = S1_CalDistor.Line_Correct2(volumetric_dict['ASCENDING'],AOI_buffer,Templist_A,'ASCENDING',
                                                volumetric_dict['ASCENDING_parms']['proj'],projScale,Origin_scale)
        LeftLayoverD,RightLayoverD,ShadowD =  S1_CalDistor.Line_Correct2(volumetric_dict['DESCENDING'],AOI_buffer,Templist_D,'DESCENDING',
                                                volumetric_dict['DESCENDING_parms']['proj'],projScale,Origin_scale)
    else:
        print('Method = {}'.format(Method))
        raise Exception('Method方法目前只有两种，请重新输入')
    # 如果不加这一段，constant会判断出错
    LeftLayoverA=LeftLayoverA.rename('LeftLayoverA');RightLayoverA=RightLayoverA.rename('RightLayoverA');ShadowA=ShadowA.rename('ShadowA')
    LeftLayoverD=LeftLayoverD.rename('LeftLayoverD');RightLayoverD=RightLayoverD.rename('RightLayoverD');ShadowD=ShadowD.rename('ShadowD')
    return LeftLayoverA,RightLayoverA,ShadowA,Templist_A,LeftLayoverD,RightLayoverD,ShadowD,Templist_D,volumetric_dict,synthesis

def Combin_AB(LeftLayoverA,RightLayoverA,ShadowA,LeftLayoverD,RightLayoverD,ShadowD,
              volumetric_dict,AOI_buffer=None,restrict=False):   
    
    def Cal_mask(LeftLayover,RightLayover,Shadow,AOI_buffer):
        # 判断是否为空
        left_empty = LeftLayover.bandNames().length().eq(0)
        right_empty = RightLayover.bandNames().length().eq(0)
        shadow_empty = Shadow.bandNames().length().eq(0)

        # 只合并非空图像
        result = ee.Image(ee.Algorithms.If(left_empty, ee.Image(), LeftLayover))
        result = ee.Image(ee.Algorithms.If(right_empty,result,result.Or(RightLayover)))
        result = ee.Image(ee.Algorithms.If(shadow_empty,result,result.Or(Shadow)))
        return result.clip(AOI_buffer)

    LeftLayoverA_,RightLayoverA_,ShadowA_ = LeftLayoverA.mask().clip(AOI_buffer),RightLayoverA.mask().clip(AOI_buffer),ShadowA.mask().clip(AOI_buffer)
    LeftLayoverD_,RightLayoverD_,ShadowD_ = LeftLayoverD.mask().clip(AOI_buffer),RightLayoverD.mask().clip(AOI_buffer),ShadowD.mask().clip(AOI_buffer)

    A_mask_ = Cal_mask(LeftLayoverA_,RightLayoverA_,ShadowA_,AOI_buffer)
    D_mask_ = Cal_mask(LeftLayoverD_,RightLayoverD_,ShadowD_,AOI_buffer)
    All_distor = A_mask_.And(D_mask_)

    DistorDict = {'LeftLayoverA':LeftLayoverA,'RightLayoverA':RightLayoverA,'ShadowA':ShadowA, 'A_mask_':A_mask_,
                  'LeftLayoverD':LeftLayoverD,'RightLayoverD':RightLayoverD,'ShadowD':ShadowD,'D_mask_':D_mask_,
                  'All_distor'  :All_distor}
    
    # 判断是否存在畸变
    A_empty = A_mask_.bandNames().contains('constant')
    A_empty = ee.Number(ee.Algorithms.If(A_empty, 1, 0))
    D_empty = D_mask_.bandNames().contains('constant')
    D_empty = ee.Number(ee.Algorithms.If(D_empty, 1, 0))

    s1_ascending_flat  = volumetric_dict['ASCENDING'].select(["VV_gamma0_flatDB","VH_gamma0_flatDB"])
    s1_descending_flat = volumetric_dict['DESCENDING'].select(["VV_gamma0_flatDB","VH_gamma0_flatDB"])
    
    if restrict:
        # 严格。如果A、D都存在畸变，则该区域为空
        s1_ascending_ =  ee.Image(ee.Algorithms.If(A_empty, s1_ascending_flat,s1_ascending_flat.updateMask(A_mask_.Not())))
        s1_descending_=  ee.Image(ee.Algorithms.If(D_empty,s1_descending_flat,s1_descending_flat.updateMask(D_mask_.Not())))                            
        Combin_AD = ee.ImageCollection([s1_ascending_,s1_descending_])
        s1_unit_mean_ = Combin_AD.mean().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
        s1_unit_max_ = Combin_AD.max().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
        s1_unit_min_ = Combin_AD.min().reproject(volumetric_dict['ASCENDING_parms']['proj']).clip(AOI_buffer)
    else:
        # 宽松，如果存在畸变，则优先采用逆轨影像替换
        s1_ascending_ = ee.Image(ee.Algorithms.If(A_empty, ee.Image(), s1_ascending_flat.where(A_mask_,s1_descending_flat)))
        s1_descending_ = ee.Image(ee.Algorithms.If(D_empty, ee.Image(), s1_descending_flat.where(D_mask_,s1_ascending_flat)))
        Combin_AD = ee.ImageCollection([s1_ascending_,s1_descending_])
        s1_unit_mean_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),s1_ascending_flat.add(s1_descending_flat).divide(2),Combin_AD.mean())).reproject(volumetric_dict['ASCENDING_parms']['proj'])

        s1_unit_max_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                    s1_ascending_flat.max(s1_descending_flat),
                    Combin_AD.max())).reproject(volumetric_dict['ASCENDING_parms']['proj'])

        s1_unit_min_ = ee.Image(ee.Algorithms.If(A_empty.Or(D_empty),
                    s1_ascending_flat.min(s1_descending_flat),
                    Combin_AD.min())).reproject(volumetric_dict['ASCENDING_parms']['proj'])
    
    return s1_unit_mean_,s1_unit_max_,s1_unit_min_,DistorDict

# 提取方法定义

## 聚类方法

In [4]:
from GEE_Extract_algorithm import Cluster_extract as Cluster

def Cluster_math(method: str, img, bands: list, index: str, visual: bool, save: bool, region=None,proj=None):
    '''method ('Kmean','SNIC','SNIC_Kmean','LVQ','Xmeans','Cobweb','CascadeKMeans')'''
    img = img.select(bands).clip(region)
    img = DataTrans.minmax_norm(img,bands,region,scale=Origin_scale,withbound=False)

    if proj:
        img = img.reproject(proj)
    if method == 'Kmean':
        result = Cluster.afn_Kmeans(img, region)
    elif method == 'Cobweb':
        result = Cluster.afn_Cobweb(img, region)
    elif method == 'Xmeans':
        result = Cluster.afn_Xmeans(img, region)
    elif method == 'LVQ':
        result = Cluster.afn_LVQ(img, region)
    elif method == 'CascadeKMeans':
        result = Cluster.afn_CascadeKMeans(img, region)
    elif method == 'SNIC':
        result = Cluster.afn_SNIC(img)
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(proj)
    elif method == 'SNIC_Kmean':
        result = Cluster.afn_SNIC(img)
        # 默认舍弃cluster和seed
        result = result.select(result.bandNames().removeAll(['clusters', 'seeds']))
        result = result.reproject(proj)
        result0 = Cluster.afn_Kmeans(result, region)  # 原始图像不参与
        result1 = Cluster.afn_Kmeans(result.addBands(img), region)  # 原始图像参与    .unmask(10)

    if visual:
        Map = geemap.Map(basemap='HYBRID')  #
        Map.centerObject(region, zoom=15)
        Map.addLayer(img.select(0), {'min': 0, 'max': 1}, 'Origin')
        if method in ['Kmean', 'Cobweb', 'Xmeans', 'LVQ', 'CascadeKMeans']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC':
            Map.addLayer(result.randomVisualizer(), {}, method)
        elif method == 'SNIC_Kmean':
            Map.addLayer(result0.randomVisualizer(), {}, 'SNIC_Kmean_NoOrigin')
            Map.addLayer(result1.randomVisualizer(), {}, 'SNIC_Kmean_YesOrigin')
        else:
            print('Please check your method str')
    else:
        Map = None
    if save:
        if method == 'SNIC_Kmean':
            DataIO.Geemap_export(filename=index + 'NoOrigin' + method + '.tif',  input=result0,
                          region=region, scale=10)
            DataIO.Geemap_export(filename=index + 'YesOrigin' + method + '.tif', input=result1,
                          region=region, scale=10)
        else:
            DataIO.Geemap_export(filename=index + method + '.tif', input=result, region=region, scale=10)
        pass
    if method == 'SNIC_Kmean':
        return Map, result0.addBands(result1)
    else:
        return Map, result

## 自适应阈值

In [5]:
from GEE_Extract_algorithm import Adaptive_threshold as Adap
import GEEMath

def Bandmath(method:str,img,band,index:str,visual:bool,save:bool,region=None,proj=None):
    '''
    method = ['otsu','minimum','yen','isodata']
    img: 仅单波段图像
    '''
    img = img.select(band).clip(region)
    img = DataTrans.minmax_norm(img,band,region,scale=Origin_scale,withbound=False)
    img = img.reproject(proj).clip(region)

    if method == 'otsu':
        histogram = GEEMath.get_histogram(img,region=region,scale=Origin_scale)
        Threshould_value = Adap.afn_otsu(histogram)
        result = img.select(0).gt(Threshould_value).clip(region)  #
        print('Threshould value is {}'.format(Threshould_value.getInfo()))

    elif method == 'minimum':
        bin_centers, counts,_  = GEEMath.get_histAndboundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_minimum(bin_centers, counts,max_num_iter = 10000)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))
        
    elif method == 'yen':
        bin_centers, counts,_ = GEEMath.get_histAndboundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_yen(bin_centers, counts)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))
    
    elif method == 'isodata':
        bin_centers, counts, bucketWidth = GEEMath.GetHistAndBoundary(img,region,Origin_scale,histNum=1000,y=300)
        Threshould_value = Adap.my_threshold_isodata(bin_centers, counts,bucketWidth)
        result = img.gt(Threshould_value).clip(region)
        print('Threshould value is {}'.format(Threshould_value))

    if visual:
        Map = geemap.Map(basemap='HYBRID') #
        Map.centerObject(region, zoom=15)
        Map.addLayer(img.select(0), {'min': 0, 'max': 1}, 'Origin')
        if method in ['otsu','minimum','yen','isodata']:
            Map.addLayer(result.randomVisualizer(), {}, method)
        else:
            print('Wrong visual! Please check your method str')
    else:
        Map = None

    if save:
        if method in ['otsu','minimum','yen','isodata']:
            DataIO.Geemap_export(filename=index+method+'.tif', input=result,region=region,scale=10)
        else:
            print('Wrong save! Please check your method str')

    return Map,result

## 监督分类

In [6]:
class Supervis_classify(object):
    pass

# 加载图像

## 选点实验

In [7]:
Glacial_lake_2015A = ee.FeatureCollection(
    'projects/ee-mrwurenzhe/assets/Glacial_lake/SAR_GLs/2019Gls_SARExt').filter(ee.Filter.gte('GL_Area', 0.1))
# 计算geometry、质心点、最小包络矩形
Geo_ext = lambda feature: feature.set({
    'Geo': feature.geometry(),
    'Centroid': feature.geometry().centroid(),
    'Rectangle': feature.geometry().bounds()})

Centrid_set = lambda feature: feature.setGeometry(feature.geometry().centroid())
Rectangle_set = lambda feature: feature.setGeometry(feature.geometry().bounds())
Glacial_lake_2015C = Glacial_lake_2015A.map(Geo_ext).map(Centrid_set)  # 添加属性,修改geometry,计算质心
Glacial_lake_2015R = Glacial_lake_2015A.map(Rectangle_set)             # 计算最小包络矩形

#抽取属性作为list
Glacial_lake_2015A_GeoList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(), ['Geo']).get('list'))
Glacial_lake_2015C_CentriodList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Centroid']).get('list'))
Glacial_lake_2015R_RectangleList = ee.List(Glacial_lake_2015C.reduceColumns(ee.Reducer.toList(),['Rectangle']).get('list'))
Num_list = Glacial_lake_2015C_CentriodList.size().getInfo()

# 选择一个时间点的影像，注意SAR和光学影像的时间点已经不一样了
# 需要放宽一些时间限制，否则可能会因为天气影响、卫星维护等问题出现数据缺失
year = '2019'
START_DATE  = ee.Date(year + '-06-01')
END_DATE   = ee.Date(year + '-08-30')
S2_START_DATE = ee.Date(year + '-06-01')
S2_END_DATE  = ee.Date(year + '-09-30')
TIME_LEN   = END_DATE.difference(START_DATE, 'days').abs()
MIDDLE_DATE  = START_DATE.advance(TIME_LEN.divide(ee.Number(2)).int(),'days')

DEMSRTM = ee.Image('USGS/SRTMGL1_003')
DEM_prj = DEMSRTM.projection()
DEMNASA = ee.Image("NASA/NASADEM_HGT/001").select('elevation')
DEMALOS = ee.ImageCollection("JAXA/ALOS/AW3D30/V3_2").mosaic().select('DSM').rename('elevation').reproject(DEM_prj)
DEMCOPERNICUS = ee.ImageCollection("COPERNICUS/DEM/GLO30").mosaic().select('DEM').rename('elevation').int16().reproject(DEM_prj)

# 地形矫正模型参数
models = ['volume', 'surface', None]     
Filter = ['boxcar','leefilter','gammamap','RefinedLee','leesigma_wrap','leesigma']
Model = models[0]
Origin_scale = 10
projScale = 30


# 选择一个点做实验
i=5 #394(面积大) #53 #11(影像都用nodata) #149(坐标系问题) #1143(畸变) #539(中等交并比)  #1296(高等交并比)

AOI_point = ee.Feature.geometry(Glacial_lake_2015C_CentriodList.get(i))
AOI_area = ee.Feature.geometry(Glacial_lake_2015A_GeoList.get(i))
AOI = ee.Feature.geometry(Glacial_lake_2015R_RectangleList.get(i))

# 计算面积
AOI_area_area = AOI_area.area().divide(ee.Number(1000*1000)).getInfo()

# 缩小
if AOI_area_area < 1:
    AOI_area_buffer = AOI_area.buffer(distance=AOI_area_area*-300)
else:
    AOI_area_buffer = AOI_area.buffer(distance=-400)

# 扩大包络矩形AOI,保证背景像素占比最大，原有基础上相应+100
if AOI_area_area < 1:
    AOI_buffer = AOI.buffer(distance=400).bounds()
else:
    AOI_buffer = AOI.buffer(distance=500).bounds()

# 载入图像，采用滤波函数，筛选日期，AOI_buffer仅用于统计是否有空值点
s1_ascending, s1_descending = load_S1collection(AOI_buffer,START_DATE,END_DATE,MIDDLE_DATE, Filter='RefinedLee',FilterSize=30)

# CLOUD_FILTER = 60           # 过滤s2 大于指定云量的数据
# CLD_PRB_THRESH = 15         # s2cloudless 概率值阈值[0-100],原实验是50
# NIR_DRK_THRESH = 0.15       # 非水暗像素判断阈值
# CLD_PRJ_DIST = 1            # 根据 CLD_PRJ_DIST 输入指定的距离从云中投射阴影
# BUFFER = 50                 # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input

# # 采用S2cloudless生产一张合成的无云图像
# s2_sr_median = merge_s2_collection(AOI_buffer,S2_START_DATE,S2_END_DATE,
#                       CLOUD_FILTER,BUFFER,CLD_PRJ_DIST,CLD_PRB_THRESH,NIR_DRK_THRESH)

Begin Filter ...


# 测试

In [8]:
# ------展示升降轨影像
# Map = geemap.Map()
# Map.centerObject(AOI_buffer, zoom=15)
# # 将GEE图层转化为ipyleaflet图层
# left_layer = geemap.ee_tile_layer(s1_ascending.select('VV').clip(AOI_buffer), {'min':-18,'max':5}, 's1_ascending')
# right_layer = geemap.ee_tile_layer(s1_descending.select('VV').clip(AOI_buffer), {'min':-18,'max':5}, 's1_descending')
# Map.split_map(left_layer, right_layer)
# Map

In [9]:
import importlib
import S1_distor_dedicated
importlib.reload(S1_distor_dedicated)
from S1_distor_dedicated import DEM_caculator,S1_CalDistor

volumetric_dict = DEM_caculator.My_s1_slope_correction(s1_ascending,s1_descending,AOI_buffer,DEMNASA,Model,Origin_scale,DistorMethed='RS')

In [10]:
Templist_A = S1_CalDistor.AuxiliaryLine2Point(volumetric_dict['ASCENDING'],volumetric_dict['ASCENDING_parms']['s1_azimuth_across'],
                            volumetric_dict['ASCENDING_parms']['coordinates_dict'],
                            volumetric_dict['ASCENDING_parms']['Auxiliarylines'], projScale)

In [11]:
def Line2Points(feature,scale=30):
    # 从Feature中提取线几何对象
    line_geometry = ee.Feature(feature).geometry()
    
    # 获取线的所有坐标点
    coordinates = line_geometry.coordinates()
    start_point = ee.List(coordinates.get(0)) # 起点坐标
    end_point = ee.List(coordinates.get(-1))  # 终点坐标
    
    # 获取线段的总长度
    length = line_geometry.length()
    num_points = length.divide(scale).subtract(1).floor()

    # 定义函数生成间隔点

    # 计算每个间隔点的坐标
    def interpolate(i):
        i = ee.Number(i)
        fraction = i.divide(num_points)
        interpolated_lon = ee.Number(start_point.get(0)).add(ee.Number(end_point.get(0)).subtract(ee.Number(start_point.get(0))).multiply(fraction))
        interpolated_lat = ee.Number(start_point.get(1)).add(ee.Number(end_point.get(1)).subtract(ee.Number(start_point.get(1))).multiply(fraction))
        return ee.Feature(ee.Geometry.Point([interpolated_lon, interpolated_lat]))
    
    # 使用map函数应用插值
    points = ee.FeatureCollection(ee.List.sequence(1, num_points).map(interpolate))
    return points

def get_neighborhood_info(point,image, neighborhood_type='4',scale = None):
    point = ee.Geometry.Point(point)
    if scale is None:
        scale = image.projection().nominalScale()
    # 根据邻域类型定义矩形的大小
    buffer_distance = scale if neighborhood_type == '4' else scale.multiply(math.sqrt(2))
    
    # 生成矩形区域
    region = point.buffer(buffer_distance).bounds()
    
    # 使用reduceRegion获取区域内的像素值
    pixel_values = image.reduceRegion(
        reducer=ee.Reducer.toList(),  # 转换为列表
        geometry=region,
        scale=scale,
        maxPixels=1e9
    )
    return pixel_values

def weighted_avg_elevation(index,points,neighbors):
    # 使用ee.Number转换index，因为原始的index是ee.ComputedObject
    index = ee.Number(index).toInt()
    point_coords = ee.List(points.get(index))
    neighbors_info = ee.Dictionary(neighbors.get(index))

    # 将邻域点信息转换为ee.Array
    x_coords = ee.Array(neighbors_info.get('longitude'))
    y_coords = ee.Array(neighbors_info.get('latitude'))
    elevations = ee.Array(neighbors_info.get('elevation'))

    # 计算距离和权重
    point1_x = ee.Number(point_coords.get(0))
    point1_y = ee.Number(point_coords.get(1))
    distances = x_coords.subtract(point1_x).pow(2).add(y_coords.subtract(point1_y).pow(2)).sqrt()
    weights = distances.pow(-1)

    # 计算加权平均高程
    weighted_elevations = elevations.multiply(weights)
    sum_weights = weights.reduce(ee.Reducer.sum(), [0])
    weighted_avg_elevation = weighted_elevations.reduce(ee.Reducer.sum(), [0]).divide(sum_weights)

    return weighted_avg_elevation

def avg_elevation(index,neighbors):
    
    # 使用ee.Number转换index，因为原始的index是ee.ComputedObject
    index = ee.Number(index).toInt()
    neighbors_info = ee.Dictionary(neighbors.get(index))
    
    elevations = ee.Array(neighbors_info.get('elevation'))
    return elevations.reduce(ee.Reducer.mean(),[0])

test = ee.List([ee.Feature(ee.Geometry.LineString([
    [93.14649155155658, 30.58754095012005],  # 起点坐标（经度，纬度）
    [93.21489203340312, 30.602349198506428]  # 终点坐标（经度，纬度）
    ])),
    ee.Feature(ee.Geometry.LineString([
    [93.14649155155658, 30.58754095012005],  # 起点坐标（经度，纬度）
    [93.21489203340312, 30.602349198506428]  # 终点坐标（经度，纬度）
    ]))])

image = ee.Image("NASA/NASADEM_HGT/001").select('elevation')  
image = image.addBands(ee.Image.pixelCoordinates(image.projection())).addBands(ee.Image.pixelLonLat()).reproject(crs=DEM_prj, scale=30)

# 使用一条线做测试
points = Line2Points(test.get(0),scale=30)  
points = points.geometry().coordinates()
neighbor_4 = points.map(partial(get_neighborhood_info,image=image,neighborhood_type='4'))
neighbor_9 = points.map(partial(get_neighborhood_info,image=image,neighborhood_type='9'))

points_elevations1 = ee.List.sequence(0, points.length().subtract(1)).map(partial(weighted_avg_elevation,points=points,neighbors=neighbor_4))
points_elevations2 = ee.List.sequence(0, points.length().subtract(1)).map(partial(avg_elevation,neighbors=neighbor_4))

In [12]:
point = ee.Geometry.Point([93.1470921, 30.58812])
image = ee.Image("NASA/NASADEM_HGT/001").select('elevation')  
image = image.addBands(ee.Image.pixelCoordinates(image.projection())).addBands(ee.Image.pixelLonLat()).reproject(crs=DEM_prj, scale=30)

buffer_region = point.buffer(30)
# 生成矩形区域
region = buffer_region.bounds()

image.reduceRegion(
    reducer=ee.Reducer.toList(),  # 转换为列表
    geometry=region,
    scale=30,
    maxPixels=1e9,
)

In [44]:
# 生成4个点
# 获取矩形的坐标
coords = ee.List(region.coordinates().get(0))

# 提取四个角点坐标
corner1 = ee.Geometry.Point(coords.get(0))
corner2 = ee.Geometry.Point(coords.get(1))
corner3 = ee.Geometry.Point(coords.get(2))
corner4 = ee.Geometry.Point(coords.get(3))

# 创建包含所有点的MultiPoint几何对象
region_coord = ee.Geometry.MultiPoint([
    corner1.coordinates(), corner2.coordinates(), 
    corner3.coordinates(), corner4.coordinates(),
])
# 使用MultiPoint几何对象作为geometry参数
result = image.reduceRegion(
    reducer=ee.Reducer.toList(),  # 转换为列表
    geometry=region_coord,
    scale=30,
    maxPixels=1e9
)
print(result.getInfo())  # 在最后使用getInfo()以在客户端打印结果

{'elevation': [4499, 4504, 4505, 4510], 'latitude': [30.587904098125374, 30.587904098125374, 30.588443087295847, 30.588443087295847], 'longitude': [93.14677171337644, 93.14731070254692, 93.14677171337644, 93.14731070254692], 'x': [979728.8781681553, 979730.818529169, 979728.8781681553, 979730.818529169], 'y': [109484.04524674865, 109484.04524674865, 109482.10488573495, 109482.10488573495]}


In [43]:
# 生成9个点

# 获取矩形的坐标
coords = ee.List(region.coordinates().get(0))

# 提取四个角点坐标
corner1 = ee.Geometry.Point(coords.get(0))
corner2 = ee.Geometry.Point(coords.get(1))
corner3 = ee.Geometry.Point(coords.get(2))
corner4 = ee.Geometry.Point(coords.get(3))

# 创建线段并计算边的中心点
edge_center1 = ee.Geometry.LineString([corner1.coordinates(), corner2.coordinates()]).centroid()
edge_center2 = ee.Geometry.LineString([corner2.coordinates(), corner3.coordinates()]).centroid()
edge_center3 = ee.Geometry.LineString([corner3.coordinates(), corner4.coordinates()]).centroid()
edge_center4 = ee.Geometry.LineString([corner4.coordinates(), corner1.coordinates()]).centroid()

# 创建包含所有点的MultiPoint几何对象
region_coord = ee.Geometry.MultiPoint([
    corner1.coordinates(), corner2.coordinates(), 
    corner3.coordinates(), corner4.coordinates(),
    edge_center1.coordinates(), edge_center2.coordinates(), 
    edge_center3.coordinates(), edge_center4.coordinates(),
    point.coordinates()
])

# 使用MultiPoint几何对象作为geometry参数
result = image.reduceRegion(
    reducer=ee.Reducer.toList(),  # 转换为列表
    geometry=region_coord,
    scale=30,
    maxPixels=1e9
)

# 结果将是一个服务器端对象
# 仅在最终结果需要时调用getInfo()
print(result.getInfo())  # 在最后使用getInfo()以在客户端打印结果

{'elevation': [4499, 4500, 4504, 4501, 4503, 4505, 4505, 4509, 4510], 'latitude': [30.587904098125374, 30.587904098125374, 30.587904098125374, 30.58817359271061, 30.58817359271061, 30.58817359271061, 30.588443087295847, 30.588443087295847, 30.588443087295847], 'longitude': [93.14677171337644, 93.1470412079617, 93.14731070254692, 93.14677171337644, 93.1470412079617, 93.14731070254692, 93.14677171337644, 93.1470412079617, 93.14731070254692], 'x': [979728.8781681553, 979729.8483486621, 979730.818529169, 979728.8781681553, 979729.8483486621, 979730.818529169, 979728.8781681553, 979729.8483486621, 979730.818529169], 'y': [109484.04524674865, 109484.04524674865, 109484.04524674865, 109483.0750662418, 109483.0750662418, 109483.0750662418, 109482.10488573495, 109482.10488573495, 109482.10488573495]}


In [ ]:
Map = geemap.Map()
Map.centerObject(point, zoom=15)
Map.addLayer(image.select('elevation').clip(point.buffer(300).bounds()).randomVisualizer(),{},'DEM')
Map.addLayer(point)
Map.addLayer(buffer_region, {'color': 'red'}, 'Buffer')
Map.addLayer(region)
Map

Map(center=[30.588120000000004, 93.1470921], controls=(WidgetControl(options=['position', 'transparent_bg'], w…

In [ ]:
Info_points = points.getInfo()
Info_neighbor_4 = neighbor_9.getInfo()

In [ ]:
def Volum_elavation(i):
    return 

def Area_elavation(i,points,info_4_neighbor):
    
    x_pred,y_pred = points[i]
    neighbors_info = info_4_neighbor[i]

    lon = neighbors_info['longitude']
    lat = neighbors_info['latitude']
    elevation = neighbors_info['elevation']
    A = np.vstack([lon, lat, np.ones(len(lon))]).T  # 构建矩阵A
    b = elevation  # 高程向量b

    # 使用最小二乘法解方程Ax = b
    x, residuals, rank, s = np.linalg.lstsq(A, b, rcond=None)

    # x包含拟合参数a, b, c（注意这里不再是1/c）
    a, b, c = x

    z_pred = a * x_pred + b * y_pred + c
    return z_pred
    

list(map(partial(Area_elavation,points=Info_points,info_4_neighbor=Info_neighbor_4),range(675)))

[4497.000000000003,
 4497.000000000003,
 4497.755129749072,
 4497.9289564666105,
 4498.600003328407,
 4500.2947452142835,
 4502.027515781112,
 4503.540822017938,
 4505.26337499544,
 4506.538725637132,
 4506.263605625543,
 4507.069686901872,
 4506.337986716884,
 4505.966436914052,
 4505.051597064245,
 4504.449242674978,
 4503.683690923208,
 4503.389080545952,
 4503.094470168755,
 4502.79985979147,
 4502.505249414156,
 4502.210639036843,
 4501.916028659616,
 4502.074604377773,
 4501.774200358486,
 4502.002910320938,
 4502.679769280599,
 4502.949172104243,
 4504.10124090279,
 4504.804866312887,
 4505.7042018037755,
 4507.054602995166,
 4508.032947897678,
 4509.226893765386,
 4510.2236606460065,
 4511.324775556335,
 4513.129029165721,
 4514.246530347737,
 4516.273911292199,
 4518.096954276785,
 4520.328673180891,
 4522.628406886943,
 4526.490241033025,
 4528.936237392481,
 4534.858160072006,
 4538.654096314684,
 4543.953200126067,
 4547.15496950224,
 4551.675172030693,
 4554.392565840855,


In [ ]:
elevation = Info_neighbor_4[0]['elevation']
lat = Info_neighbor_4[0]['latitude']
lon = Info_neighbor_4[0]['longitude']

In [ ]:
import numpy as np
from scipy.optimize import least_squares

# 数据
elevation = np.array([4500, 4497, 4497, 4500, 4497, 4497, 4501, 4500, 4499])
lat = np.array([30.587365108954902, 30.587365108954902, 30.587365108954902, 30.58763460354014, 30.58763460354014, 30.58763460354014, 30.587904098125374, 30.587904098125374, 30.587904098125374])
lon = np.array([93.14623272420602, 93.14650221879123, 93.14677171337644, 93.14623272420602, 93.14650221879123, 93.14677171337644, 93.14623272420602, 93.14650221879123, 93.14677171337644])

# 修改方程函数以匹配新的模型
def equation(params, x, y):
    a, b, c, d, e, f = params
    return a*x**2 + b*y**2 + c*x*y + d*x + e*y + f

# 修改残差函数，去除z作为变量
def residuals(params, x, y, z_true):
    z_pred = equation(params, x, y)
    return z_pred - z_true

# 初始参数猜测
initial_guess = np.zeros(6)

# 解最小二乘问题
result = least_squares(residuals, initial_guess, args=(lon, lat, elevation))

# 使用最小二乘法求得的参数
params = result.x  # 确保这里使用了之前求解得到的参数

# 根据求得的参数和给定点的经纬度计算高程
def calculate_z(params, x, y):
    # 根据新模型参数更新，只有6个参数
    a, b, c, d, e, f = params
    # 更新方程以反映新的模型形式
    z_pred = a*x**2 + b*y**2 + c*x*y + d*x + e*y + f
    return z_pred

# 给定点的经纬度
lon_pre, lat_pre = [93.14659288560377, 30.587562888265808]

# 计算给定点的预测高程z_pred
z_pred = calculate_z(params, lon_pre, lat_pre)

print(f"Predicted elevation at ({lon_pre}, {lat_pre}): {z_pred}")


Parameters: [ -272.22603537 -2061.74191617  1382.38180929  3477.35662681
  1072.72423284    75.40495244]


Predicted elevation at (93.14659288560377, 30.587562888265808): 4497.951714151349


In [ ]:
# 根据点获取面元方程，计算准确高程。接四邻域、九邻域需要本地化，插值可以在线


# 根据高程，计算几何畸变区域，并导出

## 分类方法和参数确定

In [ ]:
from PackageDeepLearn.utils import DataIOTrans as DIT
save_dir = DIT.make_dir('/root/GEE/{}补齐结果'.format(year))
os.chdir(save_dir)
Methods = ('Kmean','SNIC_Kmean','Xmeans','CascadeKMeans','otsu','isodata')
resultbands = (0,1)
Imgs = ('s1_unit_mean_',) # 's1_unit_max_','s1_unit_min_'
Bands = (['VV_gamma0_flatDB'],['VV_gamma0_flatDB','VH_gamma0_flatDB'])
mode = 'gpd'
savenmae = '{}SouthwestGLs'.format(year)

# ------------------重载变量参数
Reload = True
dest = 'users/mrwurenzhe/test'
metaName = 'test.csv'
eeUser = 'mrwurenzhe@furwas.com'
overwrite = 'yes'
delgeeUp = False    # 删除gee远端存储

# 后处理与导出